In [2]:
import ot
import numpy as np
import matplotlib.pyplot as plt

s = 0.04
n = 40000

In [3]:
def proba_rejet(u, v, W, x, y, Z, T1, T2, T3, T4):
    u,v,W = histogramme(u), histogramme(v), histogramme(W)
    x,y,Z = histogramme(x), histogramme(y), histogramme(Z)
    S1 = np.linalg.norm(T1-ot.emd(u, v, W))**2-np.linalg.norm(T1-ot.emd(x, y, Z))**2
    S2 = np.linalg.norm(T2-ot.emd(u, v, W))**2-np.linalg.norm(T2-ot.emd(x, y, Z))**2
    S3 = np.linalg.norm(T3-ot.emd(u, v, W))**2-np.linalg.norm(T3-ot.emd(x, y, Z))**2
    S4 = np.linalg.norm(T4-ot.emd(u, v, W))**2-np.linalg.norm(T4-ot.emd(x, y, Z))**2
    return 1/(2*s**2)*(S1 + S2 + S3 + S4)

def in_hypercube(u,v,W):

    if any(x < 0 or x > 1 for x in u):
        return False

    if any(x < 0 or x > 1 for x in v):
        return False

    if any(any(x < 0 or x > 1 for x in row) for row in W):
        return False
    
    return True

def histogramme(x):
    # Normaliser la liste pour que la somme soit égale à 1
    somme = np.sum(x)
    histogramme = x / somme
    return histogramme


def MH_gibbs(T1, T2, T3, T4):
    N = len(T1)
    # Initialisation
    U, V, C = np.zeros((n, N)), np.zeros((n, N)), np.zeros((n, N, N))
    U[0] = np.random.uniform(size=N)
    V[0] = np.random.uniform(size=N)
    C[0, :, :] = np.random.uniform(size=(N, N))
    def fix_coord(x):
        for i in range(N): x[i,i]=10
        return x
    # Vecteurs aléatoires
    e_u, e_v, e_C  = np.random.normal(0, 0.01, (N, n)), np.random.normal(0, 0.01, (N, n)), np.random.normal(0, 0.01, (N, N, n))
    u_u, u_v, u_c  = np.log(np.random.uniform(size=n)), np.log(np.random.uniform(size=n)), np.log(np.random.uniform(size=n))

    acceptance_rate_u = 0
    acceptance_rate_v=0
    acceptance_rate_C=0

    running_average_U = [U[0]]
    running_average_V = [V[0]]
    running_average_C = [C[0]]

    for k in range(n-1):
        # Pas pour U
        x, y, Z = U[k] + e_u[:, k], V[k], C[k]
        if not in_hypercube(x, y, Z): U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]
        else:
            if u_u[k] < proba_rejet(U[k], V[k], C[k], x, y, Z, T1, T2, T3, T4):
                U[k+1], V[k+1], C[k+1] = x, y, Z
                acceptance_rate_u += 1
            else: U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]
        UK=U[k+1]
        # Pas pour V
        x, y, Z = UK, V[k] + e_v[:, k], C[k]
        if not in_hypercube(x, y, Z): U[k+1], V[k+1], C[k+1] = UK, V[k], C[k]
        else:
            if u_v[k] < proba_rejet(UK, V[k], C[k], x, y, Z, T1, T2, T3, T4):
                U[k+1], V[k+1], C[k+1] = x, y, Z
                acceptance_rate_v += 1
            else: U[k+1], V[k+1], C[k+1] = UK, V[k], C[k]
        VK=V[k+1]
        # Pas pour C
        x, y, Z = UK, VK, C[k] + e_C[:, :, k]
        if not in_hypercube(x, y, Z):
            U[k+1], V[k+1], C[k+1] = UK, VK, C[k]
        else:
            if u_c[k] < proba_rejet(U[k], V[k], C[k], x, y, Z, T1, T2, T3, T4):
                U[k+1], V[k+1], C[k+1] = x, y, Z
                acceptance_rate_C += 1
            else: U[k+1], V[k+1], C[k+1] = UK, VK, C[k]

        # Calcul du running average
        running_average_U.append((U[k+1]+U[k])*(k/(k+1)))
        running_average_V.append((V[k+1]+V[k])*(k/(k+1)))
        running_average_C.append((C[k+1]+C[k])*(k/(k+1)))

    acceptance_rate_u /= n
    acceptance_rate_v /= n
    acceptance_rate_C /= n
    return U, V, C, acceptance_rate_u, acceptance_rate_v, acceptance_rate_C, running_average_U, running_average_V, running_average_C

In [ ]:
np.random.seed(123)

u = np.random.random(5)
v = np.random.random(5)
W = np.random.uniform(size=(5, 5))

u, v, W = histogramme(u), histogramme(v), histogramme(W)

T = ot.emd(u, v, W)

T1 = T + np.random.randn(5, 5) * s
T2 = T + np.random.randn(5, 5) * s
T3 = T + np.random.randn(5, 5) * s
T4 = T + np.random.randn(5, 5) * s

# Simulation
U, V, C, acceptance_rate_u, acceptance_rate_v, acceptance_rate_C, running_average_U, running_average_V, running_average_C = MH_gibbs(T1, T2, T3, T4)

# Trace plot pour U
plt.figure(figsize=(10, 6))
for i in range(len(U[0])):
    plt.plot(U[:, i], label=f"U[{i}]")
plt.title("Trace Plot pour U")
plt.xlabel("Iteration")
plt.ylabel("Valeur")
plt.legend()
plt.show()

# Trace plot pour V
plt.figure(figsize=(10, 6))
for i in range(len(U[0])):
    plt.plot(V[:, i], label=f"V[{i}]")
plt.title("Trace Plot pour V")
plt.xlabel("Iteration")
plt.ylabel("Valeur")
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for i in range(len(C[0])):
    plt.plot(C[:, i, 0], label=f"C[{i}][{0}]")
plt.title("Trace Plot pour C")
plt.xlabel("Iteration")
plt.ylabel("Valeur")
plt.legend()
plt.show()


print(f"Taux d'acceptation pour U : {acceptance_rate_u}")
print(f"Taux d'acceptation pour V : {acceptance_rate_v}")
print(f"Taux d'acceptation pour C : {acceptance_rate_C}")

In [34]:
print(u, running_average_U[-1]/np.sum(running_average_U[-1]))

[0.28080675 0.11536743 0.09146337 0.2222825  0.29007995] [0.29673108 0.13968273 0.082976   0.2200763  0.26053388]


In [35]:
print(v, running_average_V[-1]/np.sum(running_average_V[-1]))

[0.14285692 0.3311435  0.23122471 0.162381   0.13239387] [0.11211139 0.33981504 0.228224   0.21010884 0.10974073]


In [1]:
print(W, running_average_C[-1]/np.sum(running_average_C[-1]))

NameError: name 'W' is not defined